# Задание
Скачайте материалы к занятию. Проанализируйте ряды, сделайте их стационарными.

Решением будет ссылка на гитхаб с ноутбуком.

In [1]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from scipy.stats import boxcox

In [ ]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt

In [ ]:
robberies_in_boston = pd.read_csv("materials/Series/monthly-boston-armed-robberies-j.csv")
mean_monthly_temp = pd.read_csv("materials/Series/mean-monthly-air-temperature-deg.csv")
dowjones_closing = pd.read_csv("materials/Series/weekly-closings-of-the-dowjones-.csv")
female_births = pd.read_csv("materials/Series/daily-total-female-births-in-cal.csv")

In [ ]:
from statsmodels.tsa.stattools import adfuller
################
 # Dickey-Fuller
##################
def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    return dfoutput

In [ ]:
def tsplot(y, lags=None, figsize=(14, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        plt.figure(figsize=figsize)
        layout = (4, 1)
        ts_ax = plt.subplot2grid(layout, (0, 0), rowspan=2)
        acf_ax = plt.subplot2grid(layout, (2, 0))
        pacf_ax = plt.subplot2grid(layout, (3, 0))

        y.plot(ax=ts_ax, color='blue', label='Or')
        ts_ax.set_title('Original')

        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)

        plt.tight_layout()
    return

In [ ]:
def plot_lineplot(x, y, ds, title,  xticks_rate=10):
    sns.lineplot(x=x,y=y, data=ds)
    plt.xticks(ds[::xticks_rate].index, ds[::xticks_rate][x], rotation=30)
    plt.title(f'{title} (p-val: {test_stationarity(ds[y])["p-value"]:.3f})')

# Robberies in Boston

original

In [ ]:
ds = robberies_in_boston

In [ ]:
plot_lineplot('Month', 'Count', ds, 'Robberies in Boston')

Нормализуем дисперсию

In [ ]:
ds['Count_bc'] = boxcox(ds['Count'])[0]

In [ ]:
plot_lineplot('Month', 'Count_bc', ds, 'Robberies in Boston (after BoxCox)')

Дисперсия стала чуть более константной, но тренд остался

In [ ]:
tsplot(ds['Count_bc'])

Каких-то особых автокорреляций не видно, поэтому просто продифференцируем ряд.

In [ ]:
ds['Count_bc_diff'] = ds['Count_bc'].diff().fillna(0)

In [ ]:
plot_lineplot('Month', 'Count_bc_diff', ds, 'Robberies in Boston (after BoxCox and Diff)')

In [ ]:
test_stationarity(ds['Count_bc_diff'])

Получили стационарный ряд.

А нужна ли была стабилизация дисперсии?

In [ ]:
ds['Count_diff'] = ds['Count'].diff().fillna(0)

In [ ]:
plot_lineplot('Month', 'Count_diff', ds, 'Robberies in Boston (after Diff)')

In [ ]:
test_stationarity(ds['Count_diff'])

Судя по статистике всё хорошо. Но график показывает сильно увеличивающуюся дисперсию ряда. Так что BoxCox был необходим.

# Dowjones Closing

In [ ]:
ds = dowjones_closing

In [ ]:
plot_lineplot('Week', 'Close', ds, 'Dow-Jones Closing', xticks_rate=20)

График сильно волатильный. Посмотрим, поможет ли BoxCox.

In [ ]:
ds['Close_bc'] = boxcox(ds['Close'])[0]

In [ ]:
plot_lineplot('Week', 'Close_bc', ds, 'Dow-Jones Closing (after BoxCox)', xticks_rate=20)

Как будто бы ничего не изменилось (кроме значений ряда). Может быть есть автокорреляции?

In [ ]:
tsplot(ds['Close_bc'])

Проведём простое дифференцирование

In [ ]:
ds['Close_bc_diff'] = ds['Close_bc'].diff().fillna(0)

In [ ]:
plot_lineplot('Week', 'Close_bc_diff', ds, 'Dow Jones closing (after BoxCox and Diff)', xticks_rate=20)

In [ ]:
test_stationarity(ds['Close_bc_diff'])

Ряд проходит статистический тест на стационарность. Визуально он также похож на стационарный.